Imports

In [5]:
from pyspark.sql import SparkSession
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql.window import Window #for ranking
from pyspark.sql.functions import lit, mean, stddev_pop
from pyspark.sql.functions import collect_set, collect_list
from pyspark.sql.functions import struct
from pyspark.sql.functions import slice
from pyspark.sql.functions import col
from pyspark.sql.functions import desc
from pyspark.sql.functions import udf
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.types import DecimalType, ArrayType, IntegerType, FloatType
import findspark
import pyspark.sql.functions as F
from pyspark.sql.functions import avg, broadcast, when

Define cosine similarity and weighted avg functions

In [6]:
#defining spark 
def init_spark():
    findspark.init()
    spark = SparkSession.builder.appName('ReadMariaDB') \
    .config("spark.driver.memory", "64g") \
    .config("spark.sql.pivotMaxValues", "1000000") \
    .config("spark.jars", "D:/MariaDb/mariadb-java-client-3.1.3.jar") \
    .getOrCreate()
    return spark


# cosine similarity function
def cosine_similarity_udf(a, b):
    dot_product = sum([x * y for x, y in zip(a, b)])
    norm_a = sum([x**2 for x in a])**0.5
    norm_b = sum([x**2 for x in b])**0.5
    return dot_product / (norm_a * norm_b)


# weighted average features function
def weighted_avg_features(ratings, features):
    if not ratings or not features:
        return []

    weighted_sum = [0] * len(features[0])
    total_weight = 0

    for rating, feature in zip(ratings, features):
        weight = float(rating)
        total_weight += weight
        weighted_sum = [ws + weight * f for ws, f in zip(weighted_sum, feature)]

    if total_weight == 0:
        return weighted_sum

    return [ws / total_weight for ws in weighted_sum]

Load Dataset

In [7]:
sc = init_spark()
database = "steam_recommender"
user = "root"
password = "example"
server = "127.0.0.1"
port = 3306
jdbc_url = f"jdbc:mariadb://{server}:{port}/{database}"
jdbc_driver = "org.mariadb.jdbc.Driver"

sql = """
SELECT p.steamid, p.appid, p.playtime_2weeks, p.playtime_forever, p.dateretrieved, g.genre
FROM 01_sampled_games_2v2 AS p
JOIN games_genres AS g ON p.appid = g.appid
WHERE p.playtime_forever IS NOT NULL AND p.playtime_forever > 0
"""

df = sc.read.format("jdbc") \
    .option("url", jdbc_url) \
    .option("query", sql) \
    .option("driver", jdbc_driver) \
    .load()

# Show the data frame
df = df.drop("playtime_2weeks", "dateretrieved")

RuntimeError: Java gateway process exited before sending its port number

In [ ]:
# count number of rows in the dataframe
row_count = df.count()
# print the row count
print("Dataframe has", row_count, " rows.")
df.show()

NameError: name 'df' is not defined

Build item profiles

In [ ]:
# build the item profiles
# Group the data by 'appid' and collect the genres for each game into a list
games_genres_df = df.groupBy("appid").agg(collect_set("genre").alias("genres"))

# Create a list of unique genres
unique_genres = sorted(df.select("genre").distinct().rdd.flatMap(lambda x: x).collect())

# Define a UDF to create a binary vector for each game's genres
@udf(returnType=ArrayType(IntegerType()))
def genre_vector(genres):
    return [1 if genre in genres else 0 for genre in unique_genres]
# Add a new column 'genre_vector' to the DataFrame
# the genre vector will now have a 1 for each genre that the game belongs to
games_genres_df = games_genres_df.withColumn("genre_vector", genre_vector("genres"))

# games_genres_df.show(truncate=False)
# Join the main DataFrame with the games_genres_df on appid to include the genre_vector
df = df.join(broadcast(games_genres_df.select("appid", "genre_vector")), on="appid")
df.show()

+-----+-----------------+----------------+----------+--------------------+
|appid|          steamid|playtime_forever|     genre|        genre_vector|
+-----+-----------------+----------------+----------+--------------------+
|  300|76561197960268000|             109|    Action|[1, 0, 0, 0, 0, 0...|
| 1300|76561197960268000|              94|    Action|[1, 0, 0, 0, 0, 0...|
| 2100|76561197960268000|             110|    Action|[1, 0, 0, 0, 0, 0...|
| 2100|76561197960268000|             110|       RPG|[1, 0, 0, 0, 0, 0...|
| 4000|76561197960268000|             152|     Indie|[0, 0, 0, 0, 0, 0...|
| 4000|76561197960268000|             152|Simulation|[0, 0, 0, 0, 0, 0...|
| 2600|76561197960268000|              59|    Action|[1, 0, 0, 0, 0, 0...|
| 9000|76561197960268000|               2|    Action|[1, 0, 0, 0, 0, 0...|
| 2300|76561197960268000|              40|    Action|[1, 0, 0, 0, 0, 0...|
| 2200|76561197960268000|             210|    Action|[1, 0, 0, 0, 0, 0...|
| 4500|76561197960268000|

Build the user profile

In [ ]:
# 1. Calculate the global playtime average for each game
global_playtime_avg = df.groupBy("appid").agg(avg("playtime_forever").alias("global_playtime_avg"))

# 2. Normalize the user's playtime for each game based on the global average
df = df.join(broadcast(global_playtime_avg), on="appid")
df = df.withColumn("playtime_normalized", F.when(df.playtime_forever == 0, 1).otherwise(df.playtime_forever / df.global_playtime_avg))

# 3. Implement the user profile
# First, let's group the data by user and aggregate the genre vectors and normalized playtimes
user_aggregated_data = df.groupBy("steamid").agg(
    collect_list("genre_vector").alias("genres_list"),
    collect_list("playtime_normalized").alias("playtime_normalized_list")
)

# Now, let's define a UDF to calculate the weighted average of genre vectors
weighted_avg_features_udf = udf(weighted_avg_features, ArrayType(FloatType()))

# Calculate the user profile as the weighted average of rated item profiles (genre vectors)
user_profiles = user_aggregated_data.withColumn("user_profile", weighted_avg_features_udf("playtime_normalized_list", "genres_list"))

user_profiles.show()

+-----------------+--------------------+------------------------+--------------------+
|          steamid|         genres_list|playtime_normalized_list|        user_profile|
+-----------------+--------------------+------------------------+--------------------+
|76561197960268000|[[0, 0, 0, 1, 0, ...|    [1.0, 1.0, 1.0, 1...|[0.69411767, 0.21...|
|76561197960274000|[[1, 0, 0, 0, 0, ...|    [1.0, 1.0, 1.0, 1...|[0.77272725, 0.04...|
|76561197960277000|[[1, 0, 0, 0, 0, ...|              [1.0, 1.0]|[1.0, 0.0, 0.0, 0...|
|76561197960283000|[[1, 0, 0, 1, 0, ...|    [1.0, 1.0, 1.0, 1...|[0.375, 0.125, 0....|
|76561197960288000|[[1, 0, 0, 0, 0, ...|                   [1.0]|[1.0, 0.0, 0.0, 0...|
|76561197960293000|[[0, 0, 0, 1, 0, ...|    [1.0, 1.0, 1.0, 1...|[0.21428572, 0.21...|
|76561197960303000|[[1, 0, 0, 0, 0, ...|                   [1.0]|[1.0, 0.0, 0.0, 0...|
|76561197960306000|[[1, 0, 0, 0, 0, ...|    [1.0, 1.0, 1.0, 1...|[0.5744681, 0.276...|
|76561197960324000|[[1, 0, 0, 0, 0, ...|   

Cosine similarity

In [ ]:
# prediction heuristics
# calculate cosine distance of an item and user profile

# 1. create udf for cosine similarity
cosine_similarity = udf(cosine_similarity_udf, FloatType())
# cross join the game_genres_df with the user_profiles
cross_joined = games_genres_df.crossJoin(user_profiles)

# calculate the cosine similarity between each item and user
recommendations = cross_joined.withColumn(
    "similarity", cosine_similarity("genre_vector", "user_profile")
)

# sort based on similarity score
sorted_recommendations = recommendations.sort(desc("similarity"))

sorted_recommendations.show(10)

+-----+---------------+--------------------+-----------------+--------------------+------------------------+--------------------+----------+
|appid|         genres|        genre_vector|          steamid|         genres_list|playtime_normalized_list|        user_profile|similarity|
+-----+---------------+--------------------+-----------------+--------------------+------------------------+--------------------+----------+
| 4300|[Action, Indie]|[1, 0, 0, 0, 0, 0...|76561197960288000|[[1, 0, 0, 0, 0, ...|                   [1.0]|[1.0, 0.0, 0.0, 0...|       1.0|
|42500|[Action, Indie]|[1, 0, 0, 0, 0, 0...|76561197960288000|[[1, 0, 0, 0, 0, ...|                   [1.0]|[1.0, 0.0, 0.0, 0...|       1.0|
|19200|[Action, Indie]|[1, 0, 0, 0, 0, 0...|76561197960288000|[[1, 0, 0, 0, 0, ...|                   [1.0]|[1.0, 0.0, 0.0, 0...|       1.0|
|22200|[Action, Indie]|[1, 0, 0, 0, 0, 0...|76561197960288000|[[1, 0, 0, 0, 0, ...|                   [1.0]|[1.0, 0.0, 0.0, 0...|       1.0|
|26000|[Actio

recommender

In [ ]:
# Create a window by steamid and similarity to get ranking
window_spec = Window.partitionBy("steamid").orderBy(desc("similarity"))

ranked_recommendations = sorted_recommendations.withColumn("rank", F.row_number().over(window_spec))

top_10_recommendations = ranked_recommendations.filter(ranked_recommendations.rank <= 10)
top_10_recommendations.show(10)

NameError: name 'Window' is not defined